In [1]:
!pip install --upgrade optimum transformers torch

  Using cached transformers-4.43.4-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.43.4-py3-none-any.whl (9.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [2]:
!pip install optimum-intel

In [3]:
!pip install onnx onnxruntime onnxruntime-tools transformers

In [4]:
pip install --upgrade torch torchvision onnx onnxruntime transformers

  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.0-py3-none-any.whl (9.5 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.4
    Uninstalling transformers-4.43.4:
      Successfully uninstalled transformers-4.43.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optimum 1.21.3 requires transformers[sentencepiece]<4.44.0,>=4.29.0, but you have transformers 4.44.0 which is incompatible.


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Create dummy input
dummy_input = tokenizer("This is a test.", return_tensors="pt")
input_ids = dummy_input["input_ids"]
attention_mask = dummy_input["attention_mask"]

# Export the model with opset version 14
torch.onnx.export(
    model,
    (input_ids, attention_mask),
    "model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["output"],
    opset_version=14,  # Updated opset version
    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}, "output": {0: "batch_size"}}
)


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Quantize the ONNX model
quantize_dynamic(
    "model.onnx",
    "quantized_model.onnx",
    weight_type=QuantType.QInt8
)
